In [1]:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ candidates snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from snorkel.lf_helpers import get_aligned_ngrams
def LF_aligned_temp_word(c):
    ngrams = list(get_aligned_ngrams(c.temp))
    if ((c[0].parent.document.name).upper(),
            (c[0].get_span()).upper(),
            (''.join(c[1].get_span().split())).upper()) == ('RECTS01158-1', 'BC807-25', '-55'):
        import pdb; pdb.set_trace()
    return 1 if ('storage' in ngrams or
                 'temperature' in ngrams or
                 'tstg' in ngrams or
                 'ts' in ngrams or
                 'stg' in ngrams) else 0

In [4]:
from snorkel.models import CandidateSet
candidates = session.query(CandidateSet).all()[0]
print len(candidates)

85377


In [11]:
%%time
from snorkel.utils import ProgressBar
pb = ProgressBar(len(candidates))
neg = set()
pos = set()
for i, c in enumerate(candidates):
    pb.bar(i)
    if LF_aligned_temp_word(c):
        pos.add(c)
    else:
        neg.add(c)
pb.close()

[===================                     ] 45%> <ipython-input-10-7b4088d81156>(8)LF_aligned_temp_word()
-> return 1 if ('storage' in ngrams or
(Pdb) print ngrams
[u'-150', u'-150']
(Pdb) q


BdbQuit: 

In [6]:
print len(pos)

26088


In [8]:
from snorkel.models import Corpus
from snorkel.utils import get_ORM_instance
from hardware_utils import entity_level_f1

tp = pos
fp = set()
tn = set()
fn = set()

corpus = get_ORM_instance(Corpus, session, 'Hardware Training')
gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(TP, FP, FN) = entity_level_f1(tp, fp, tn, fn, gold_file, corpus, 'stg_temp_min')

Scoring on Entity-Level Gold Data
Corpus Precision 1.0
Corpus Recall    0.999
Corpus F1        0.999
----------------------------------------
TP: 669 | FP: 0 | FN: 1



In [9]:
from pprint import pprint
pprint(FN)

set([('RECTS01158-1', 'BC807-25', '-55')])
